In [ ]:
# #legacy code from the original chatbot

# def get_chatGPT_response(query: str, context: List[str]) -> str:
#     """
#     Queries the GPT API to get a response to the question.

#     Args:
#     query (str): The original query.
#     context (List[str]): The context of the query, returned by embedding search.

#     Returns:
#     A response to the question.
#     """
#     response = openai.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=build_prompt(query, context),
#     )

#     return response.choices[0].message.content  # type: ignore


# def build_prompt(query: str, context: List[str]) -> List[ChatCompletionMessageParam]:
#     """
#     Builds a prompt for the LLM. #

#     This function builds a prompt for the LLM. It takes the original query,
#     and the returned context, and asks the model to answer the question based only
#     on what's in the context, not what's in its weights.

#     More information: https://platform.openai.com/docs/guides/chat/introduction

#     Args:
#     query (str): The original query.
#     context (List[str]): The context of the query, returned by embedding search.

#     Returns:
#     A prompt for the LLM (List[ChatCompletionMessageParam]).
#     """

#     system: ChatCompletionMessageParam = {
#         "role": "system",
#         "content": "I am going to ask you a question, which I would like you to answer"
#         "based only on the provided context, and not any other information."
#         "If there is not enough information in the context to answer the question,"
#         'say "I am not sure", then try to make a guess.'
#         "Break your answer up into nicely readable paragraphs.",
#     }
#     user: ChatCompletionMessageParam = {
#         "role": "user",
#         "content": f"The question is {query}. Here is all the context you have:"
#         f'{(" ").join(context)}',
#     }

#     return [system, user]



In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')

import os
pdf_list=os.listdir("/Users/matansharon/python/opensource_rag/pdf_files")
text_list=os.listdir("/Users/matansharon/python/opensource_rag/text_files")

for pdf in pdf_list:
    temp=pdf.replace(".pdf",".txt")
    if temp not in text_list:
        text=convert_from_pdf_to_text("pdf_files/"+pdf)
        with open("text_files/"+temp, 'w') as f:
            f.write(text)

In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


In [ ]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)


In [ ]:
import chromadb
persist_directory = "chroma_storage"
client = chromadb.PersistentClient(path=persist_directory)
collection=client.get_collection("file1_collection")
print(collection.get().keys())



In [ ]:
results

In [ ]:
results = collection.query(
            query_texts=["summerize the Component Separation method for CMB using Convolutional Neural Networks document"], n_results=5, include=["documents", "metadatas","distances"])
print(results)
for i in range(len(results['distances'][0])):
    if (abs(results['distances'][0][i]-1)<0.4):
        print(results['distances'][0][i])
        print(results['documents'][0][i][:100])
        print(results['metadatas'][0][i])
        print("-"*100)

In [ ]:
#this cell is a testing cell for filtering the results of the query by the distance

temp_results={"ids":[[]],"distances":[[]],"metadatas":[[]],"documents":[[]]}
        for i in range(5):
            if abs(1-results["distances"][0][i]<0.4): 
                temp_results["ids"][0].append(results["ids"][0][i])
                temp_results["distances"][0].append(results["distances"][0][i])
                temp_results["metadatas"][0].append(results["metadatas"][0][i])
                temp_results["documents"].append(results["documents"][0][i])
        results=temp_results

In [ ]:
all_file_names=set()
for line in collection.get()['metadatas']:
    all_file_names.add(line['filename'])
all_file_names

In [ ]:
for doc in collection.get()['documents']:
    print(len(doc))

In [ ]:
import PyPDF2
pages=PyPDF2.PdfReader("/Users/matansharon/python/opensource_rag/pdf_files/qlora.pdf").pages


In [ ]:
# from nltk.tokenize import word_tokenize
# for page in pages:
#     print(len(word_tokenize(page.extract_text())))

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
df=pd.read_csv("salaries.csv")

df_str=df.to_string().strip()
len(word_tokenize(df_str))
with open("salaries2.txt", 'w') as f:
    f.write(df_str)

In [ ]:
all_rows=(df_str.split("\n"))
col_rows_names=all_rows[0]
# print(col_rows_names.strip())
for line in all_rows:
    print((line))

In [ ]:
import anthropic
import os
from dotenv import load_dotenv
load_dotenv()

client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
message = client.messages.create(
        # model="claude-3-opus-20240229",
        model='claude-3-haiku-20240307',
        max_tokens=4000,
        temperature=0.2  ,
        system=f"""
        You are a data analyst assistant.
        your given a dataframe that was converted into string with salaries of employees,
        return the head of the dataframe.
        
        """,
        messages=[
            {"role": "user", "content": df_str}
        ]
    )
print(message.usage)
print( message.content[0].text)



In [ ]:
message

In [ ]:
class Message():
    def __init__(self):
        self.content=''
        self.type=''
    def get_content(self):
        return self.content
    def get_type(self):
        return self.type
        

In [ ]:
import matplotlib.pyplot as plt

# Assuming the dataframe is named 'df'
plt.figure(figsize=(12, 6))
plt.hist(df1000['salary'], bins=20)
plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.title('Salary Distribution')
plt.show()

In [113]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
# nltk.download('punkt')

files=os.listdir("/Users/matansharon/python/opensource_rag/documents")
files
for file in files:
    if file.endswith(".pdf"):
        print(file)
        pages=convert_from_pdf_to_text("/Users/matansharon/python/opensource_rag/documents/"+file)
        print(len(word_tokenize(pages)))
    elif file.endswith(".csv"):
        df=pd.read_csv("/Users/matansharon/python/opensource_rag/documents/"+file)
        df_str=df.to_string().strip()
        print(df_str)
        # print(len(word_tokenize(df_str)))

work_year experience_level employment_type                               job_title   salary salary_currency  salary_in_usd employee_residence  remote_ratio company_location company_size
0         2024               MI              FT                          Data Scientist   120000             USD         120000                 AU             0               AU            S
1         2024               MI              FT                          Data Scientist    70000             USD          70000                 AU             0               AU            S
2         2024               MI              CT                          Data Scientist   130000             USD         130000                 US             0               US            M
3         2024               MI              CT                          Data Scientist   110000             USD         110000                 US             0               US            M
4         2024               MI              FT   

In [141]:
def read_and_convert_csv(file_path: str) -> str:
    df=pd.read_csv(file_path)
    # Convert the dataframe to a string
    df=df.to_string().strip()
    column_row=df.split("\n")[0]
    #split the string into a list of strings
    rows=df.split("\n")
    pages=[]
    page=[]
    for i in range(1,len(rows),10):
        page.append(column_row)
        for j in range(i,i+10):
            if j<len(rows):
            
                page.append(rows[j])
        pages.append(page)
        page=[]
    return pages
res=read_and_convert_csv("/Users/matansharon/python/opensource_rag/documents/salaries.csv")
for page in res:
    for row in page:
        print(row)


work_year experience_level employment_type                               job_title   salary salary_currency  salary_in_usd employee_residence  remote_ratio company_location company_size
0         2024               MI              FT                          Data Scientist   120000             USD         120000                 AU             0               AU            S
1         2024               MI              FT                          Data Scientist    70000             USD          70000                 AU             0               AU            S
2         2024               MI              CT                          Data Scientist   130000             USD         130000                 US             0               US            M
3         2024               MI              CT                          Data Scientist   110000             USD         110000                 US             0               US            M
4         2024               MI              FT   

In [114]:
splited=df_str.split('\n')
for row in splited:
    print(row)

work_year experience_level employment_type                               job_title   salary salary_currency  salary_in_usd employee_residence  remote_ratio company_location company_size
0         2024               MI              FT                          Data Scientist   120000             USD         120000                 AU             0               AU            S
1         2024               MI              FT                          Data Scientist    70000             USD          70000                 AU             0               AU            S
2         2024               MI              CT                          Data Scientist   130000             USD         130000                 US             0               US            M
3         2024               MI              CT                          Data Scientist   110000             USD         110000                 US             0               US            M
4         2024               MI              FT   

In [ ]:


anthropic_client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
context=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/documents/Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf')
query='summerize this book'
response = anthropic_client.messages.create(
        # model="claude-3-opus-20240229",
        model='claude-3-haiku-20240307',
        max_tokens=1000,
        temperature=0.2,
        system=f"""
        I am going to ask you a question, which I would like you to answer"
        "based only on the provided context, and not any other information."
        "If there is not enough information in the context to answer the question,"
        'say "I am not sure", then try to make a guess.'
        "Break your answer up into nicely readable paragraphs.
        here is all the context you have:{context}"
        """,
        messages=[
            {"role": "user", "content": query}
        ]
    )

In [ ]:
print(response.content[0].text)


In [ ]:
response.usage

In [ ]:
110840/80055

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')
model

In [ ]:
%%time
response = model.generate_content("What is the meaning of life?")

In [39]:
import chromadb
db_client = chromadb.PersistentClient(path='chroma_storage_pages')
collection=db_client.get_collection('by_pages_collection')
results = collection.query(
            query_texts=['what is qlora?'], n_results=3, include=["documents", "metadatas","distances"],
        )
        
    
context=results["documents"][0]


In [42]:
context

['Figure 1: Different finetuning methods and their memory requirements. QLORAimproves over LoRA by\nquantizing the transformer model to 4-bit precision and using paged optimizers to handle memory spikes.\n2 Background\nBlock-wise k-bit Quantization Quantization is the process of discretizing an input from a rep-\nresentation that holds more information to a representation with less information. It often means\ntaking a data type with more bits and converting it to fewer bits, for example from 32-bit floats to\n8-bit Integers. To ensure that the entire range of the low-bit data type is used, the input data type is\ncommonly rescaled into the target data type range through normalization by the absolute maximum\nof the input elements, which are usually structured as a tensor. For example, quantizing a 32-bit\nFloating Point (FP32) tensor into a Int8 tensor with range [−127,127]:\nXInt8=round\x12127\nabsmax (XFP32)XFP32\x13\n=round (cFP32·XFP32), (1)\nwhere cis the quantization constant or

In [35]:
import os
import PyPDF2
reader=PyPDF2.PdfReader("/Users/matansharon/python/opensource_rag/pdf_files/Non-linear Welfare-Aware Strategic Learning.pdf")
pages=reader.pages
text=""
for page in pages:
    text+=(page.extract_text())
text

'Non-linear Welfare-Aware Strategic Learning\nTian Xie1and Xueru Zhang1\n{xie.1379, zhang.12807}@osu.edu\n1the Ohio State University\nApril, 2024\nAbstract\nThis paper studies algorithmic decision-making in the presence of strategic individual be-\nhaviors, where an ML model is used to make decisions about human agents and the latter can\nadapt their behavior strategically to improve their future data. Existing results on strategic\nlearning have largely focused on the linearsetting where agents with linear labeling function\nbest respond to a (noisy) linear decision policy. Instead, this work focuses on general non-linear\nsettings where agents respond to the decision policy with only "local information" of the policy.\nMoreover, we simultaneously consider objectives of maximizing decision-maker welfare (model\nprediction accuracy), social welfare (agent improvement caused by strategic behaviors), and\nagent welfare (the extent that ML underestimates the agents). We first generalize t

In [14]:
import nltk
from nltk.tokenize import word_tokenize
len(word_tokenize(text))


7660

In [36]:
import google.generativeai as genai
import os
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')


response = model.generate_content(f"summerize the following: {text}")
print(response.text)

This paper presents a comprehensive analysis of the relationships between the welfare of the decision-maker, agent, and society in non-linear strategic learning settings. The authors extend the agent response model to allow agents to respond strategically based on local information and knowledge about the decision policy. They analyze the compatibility of different welfare objectives under general non-linear settings and provide a general optimization framework for balancing the welfare of all parties. This work provides new insights into the challenges and opportunities in designing welfare-aware machine learning algorithms for strategic learning settings.


In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import os
import nltk

# nltk.download('punkt')
path="/Users/matansharon/python/opensource_rag/נתוני חברים פברואר - אפריל 2024.xlsx"
df=pd.read_excel(path,sheet_name='גיליון3')
# print(df.to_string())
print("length of tokens of sheet 3: ",len(word_tokenize(df.to_string()))*1.3)




length of tokens of sheet 3:  3730


In [16]:
import anthropic
client=anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"))
message = client.messages.create(
        # model="claude-3-opus-20240229",
        model='claude-3-haiku-20240307',
        max_tokens=4000,
        temperature=0.2,
        system=f"""
        You are a data analyst assistant.
        your given a dataframe that was converted into string with salaries of employees,
        return the head of the dataframe.
        
        """,
        messages=[
            {"role": "user", "content": df.to_string()}
        ]
    )